In [8]:
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np

# 文件夹路径
folder_path = ['/kaggle/input/food11/food-11/training',
               '/kaggle/input/food11/food-11/validation',
               '/kaggle/input/food11/food-11/testing']

train_foods = []  # 整形
train_images = []  # numpy.ndarray类型
valid_foods = []
valid_images = []
test_images = []
test_pic_names = []

class_names = ['Bread', 
               'Dairy product', 
               'Dessert', 
               'Egg',
               'Fried food', 
               'Meat', 
               'Noodles/Pasta', 
               'Rice', 
               'Seafood', 
               'Soup', 
               'Vegetables/Fruits']
class_names_cn = ['面包', 
                  '乳制品', 
                  '甜点',
                  '鸡蛋', 
                  '油炸食品',
                  '肉类', 
                  '面条/意大利面',
                  '米饭', 
                  '海鲜', 
                  '汤', 
                  '蔬菜/水果']

# 遍历文件夹中所有文件
for i in range(2):
    for file_name in os.listdir(folder_path[i]):
        file_path = folder_path[i] + '/' + file_name
        if i==1:
            # 验证集
            valid_foods.append(int(file_name.split('_')[0]))
            valid_images.append(cv2.imread(file_path))
        # 训练集
        train_foods.append(int(file_name.split('_')[0]))
        train_images.append(cv2.imread(file_path))

for file_name in os.listdir(folder_path[2]):
    test_pic_names.append(('/food11/testing/'+file_name))
    file_path = folder_path[2] + '/' + file_name
    test_images.append(cv2.imread(file_path))

KeyboardInterrupt: 

In [ ]:
import torchvision.transforms as transforms
plt.imshow(train_images[0][:,:,::-1])
# [:,:,::-1]用于反转img数组的颜色通道，以便正确显示图像
# opencv 是BGR, matplotlib是RGB
plt.savefig('/kaggle/working/1.png', bbox_inches='tight', dpi=800)
plt.show()
print(train_images[0].shape)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
#图像增强
# pytorch是NCHW
train_transform = transforms.Compose([
    transforms.ToPILImage(), # 转化为PIL类型便于进行Resize
    transforms.Resize((128, 128)),
    transforms.ColorJitter(brightness=0.5, contrast=0.5),#随机调整亮度和对比度
    transforms.RandomHorizontalFlip(),#随机将图片水平翻转
    transforms.RandomRotation(15),#随机旋转图片
    transforms.ToTensor(),#将图片转化为Tensor(GBR->NCHW)
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) #归一化到-1dao1
])

#测试集图像处理
valid_transform = transforms.Compose([
    transforms.ToPILImage(), # 转化为PIL类型便于进行Resize    
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) #归一化到-1 1
    
])

In [ ]:
import random
# a为随机从train_images中取出的4个值
test_a = random.sample(train_images, 4)

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

# 创建一个4x2的子图，并在每个子图中显示一张图片
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(8, 16))
for i, ax in enumerate(axes.flat):
    if (i % 2==0) or (i == 0):
        a = test_a[i//2]
    else:
        a = train_transform(test_a[i//2])
        # 反标准化
        a = a * torch.tensor(std).view(3, 1, 1) + torch.tensor(mean).view(3, 1, 1)
        # Tensor转换为Numpy
        a = a.numpy()
        # CHW->HWC
        a = np.transpose(a, (1, 2, 0))
    a = a[:, :, ::-1] # 将每个通道的值逆序排列 BGR to RGB
    
    # 显示图片
    ax.imshow(a)
    ax.axis('off')
plt.savefig('/kaggle/working/2.png', bbox_inches='tight', dpi=800)
plt.show()

In [ ]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, features, labels=None, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        feature = self.features[index]
        if self.labels:
            label = self.labels[index]
        else:
            label = None
        # 应用转换操作
        if self.transform:
            feature = self.transform(feature)
        return feature, label

In [ ]:
from torch.utils.data import DataLoader

train_data = MyDataset(features = train_images,
                       labels = train_foods,
                       transform = train_transform)
valid_data = MyDataset(features = valid_images,
                       labels = valid_foods,
                       transform = valid_transform)
train_set = DataLoader(dataset = train_data,
                       batch_size = 8,
                       shuffle = True,
                       drop_last = True)
valid_set = DataLoader(dataset = valid_data,
                      batch_size = 8,
                      shuffle = True,
                      drop_last = True)

In [ ]:
test_set = MyDataset(features = test_images, transform=valid_transform)

In [ ]:
import torch
dvc = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(dvc)

In [ ]:
!pip install torchsummary

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

# 定义残差块
class Residual(nn.Module):
    def __init__(self, input_channels, out_channels, use_1x1conv = False, strides = 1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, out_channels, kernel_size = 3, padding = 1, stride = strides)
        # x-3+2/s +1 = x  s=2,减半
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        if use_1x1conv: #1x1卷积改变x维度，使得其与卷积后的y维度一致，可以相加
            self.conv3 = nn.Conv2d(input_channels, out_channels, kernel_size = 1, stride = strides)
        else:
            self.conv3 = None
    def forward(self, x):
        y = F.relu(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))
        if self.conv3:
            x = self.conv3(x) #改变维度
        y += x
        return F.relu(y)

# 生成残差块
def resnet_block(input_channels, out_channels, num_residuals,
                 half_wh=False):
    blk = []
    for i in range(num_residuals):
        # half_wh就高宽减半(strides=2)，否则不变
        if i == 0 and not half_wh:
            blk.append(Residual(input_channels, out_channels,
                                use_1x1conv=True, strides=1))
        elif i == 0 and half_wh:
            blk.append(Residual(input_channels, out_channels,
                                use_1x1conv=True, strides=2))
        else: 
            blk.append(Residual(out_channels, out_channels))
    return blk



class qyeNet(nn.Module):
    def __init__(self):
        super(qyeNet, self).__init__()
        self.conv_pool1 = nn.Sequential(
            # 输入 224*224*3
            *resnet_block(3, 64, 1),
            # 输出224 224 64
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出(224-2)/2 +1 = 112 112 64
        )
        self.conv_pool2 = nn.Sequential(
            # 输入 112 112 64
            *resnet_block(64, 128, 1),
            # 输出 112 112 128
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出 56 56 128
        )
        self.conv_pool3 = nn.Sequential(
            *resnet_block(128, 256, 2),
            # 输出 56
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出 28 28 256
        )
        self.conv_pool4 = nn.Sequential(
            *resnet_block(256, 512, 2),
            # 输出28 28 512
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出14 14 512
        )
        self.conv_pool5 = nn.Sequential(
            *resnet_block(512, 512, 2),
            # 输出 14 14 512
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出 7 7 512
        )
        self.fc = nn.Sequential(
            nn.Linear(7*7*512, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 11)
            # 使用交叉熵误差函数，最后一层不用softmax
        )
    def forward(self, x):
        x = self.conv_pool1(x)
        x = self.conv_pool2(x)
        x = self.conv_pool3(x)
        x = self.conv_pool4(x)
        x = self.conv_pool5(x)
        x = x.view(-1, 7*7*512)
        x = self.fc(x)
        return x

model_qye = qyeNet().to(dvc)
summary(model_qye, (3, 224, 224))

In [ ]:
def valid(model, valid_set):
    model.eval()
    valid_correct = 0
    for i, data in enumerate(valid_set, 0):
        data_len = len(data[0])
        features, labels = data
        features, labels = features.to(dvc), labels.to(dvc)
        preds = model(features)
        _, ans = torch.max(preds, 1) # 返回1维度上的最大值的索引，_是最大值
        valid_correct += torch.sum(ans==labels.data)
    valid_accuracy = 100*valid_correct/(len(valid_set)*data_len)
    print(f'Accuracy on valid set:{valid_accuracy:.3f}%')
    return valid_accuracy

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from datetime import timedelta
from datetime import timezone
import time, math
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, StepLR

SHA_TZ = timezone(
    timedelta(hours=8),
    name='Asia/Shanghai',
)

def get_cos(t):
    return 0.0051 + 0.0049*math.cos(2*math.pi*t/10)

def train_valid(model, train_set, valid_set):
    writer = SummaryWriter('/kaggle/working/runs/vgg') # 开始记录
    loss_func = nn.CrossEntropyLoss() #交叉熵损失
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #优化器以及学习率
    epochs = 100
    start_time = time.time()
    good_num = 0 # 学习率大于85%的次数或模型过拟合的次数
    for epoch in range(epochs):
        lr = get_cos(epoch)
        print(lr)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr) #优化器以及学习率
        if good_num >= 5:
            print(f'run {epoch} epoch')
            break
        model.train()
        sum_loss = 0
        train_correct = 0
        time_now = datetime.utcnow().astimezone(SHA_TZ)
        print(f'[{epoch+1}/{epochs}]  time now: {time_now.strftime("%Y-%m-%d %H:%M:%S")}')
        for i, data in enumerate(train_set, 0):
            features, labels = data
            data_len = len(data[0])
            features, labels = features.to(dvc), labels.to(dvc)
            preds = model(features)
            model.zero_grad() #梯度清零
            # 计算损失
            loss = loss_func(preds, labels)
            loss.backward() # 反向传播
            optimizer.step()
#             scheduler.step()
            sum_loss += loss.data
            _, ans = torch.max(preds, 1) # 此时preds第0维度有5个数据，ans也是5*1的张量
            train_correct += torch.sum(ans==labels.data) # ans5*1 ，这里输出的是5个里面预测准确的数量
        train_ac = 100*train_correct/(len(train_set)*data_len) # 训练集准确率
        train_loss = sum_loss/(len(train_set)*data_len) # 训练集损失
        print(f'Loss on train set: {train_loss:.3f} Accuracy on train set: {train_ac:.3f}%')
        valid_ac = valid(model, valid_set) # 测试集准确率(有输出)
        if valid_ac > 90 or train_ac > 99:
            good_num += 1
        elif valid_ac == 95:
            good_num = 5
        writer.add_scalar("Accuracy/train", train_ac, epoch)
        writer.add_scalar("Loss/train", train_loss, epoch)
        writer.add_scalar("Accuracy/valid", valid_ac, epoch)
    end_time = time.time()
    print(f'cost time {end_time - start_time}')
    writer.close() # 结束记录

In [ ]:
train_valid(model_qye, train_set, valid_set)

In [ ]:
torch.save(model_qye, '/kaggle/working/qyecos.pth')

In [ ]:
import matplotlib.pyplot as plt
def predict(model, data_set, length, prt_info=True):
    model.eval()
    model.cpu()
    '''
    data：DataLoader类型
    length: 预测数据批量
    prt_info: 是否打印预测细节
    '''
    data_iter = iter(data_set) # 定义迭代器去迭代数据
    pre_correct = 0
    true_labels = [] # 真实标签集合
    pred_labels = [] # 预测标签集合
    pred_ans = [] #1代表正确，0错误
    for i in range(length):
        try: # 防止迭代溢出
            data = next(data_iter)
        except StopIteration:
            break #长度超了就跳出循环
        feature, label = data
        # 预测并打印预测标签
        feature = feature.cpu()
        if label != None:
            label = label.cpu()
            data_len = len(data[0])
        else:
            feature = feature.reshape((1, 3, 224, 224))
            data_len = 1
        pre_tensor = model(feature)
        _, pre_label = torch.max(pre_tensor, 1)
        if label != None:
            pre_correct += torch.sum(pre_label==label) # 5*1
        if prt_info:
            # 为可视化准备
            fig, ax = plt.subplots(1, data_len, figsize=(15, 3))  # 创建1行x列的子图网格
        for j in range(data_len):
            if prt_info: #要输出信息时
            # 可视化图片
            # 将通道维度调整到最后一个维度
            
                f_np = feature[j].cpu() * torch.tensor(std).view(3, 1, 1) + torch.tensor(mean).view(3, 1, 1)
                f_np = f_np.numpy()
                f_np = np.transpose(f_np, (1, 2, 0))
                f_np = f_np[:, :, ::-1]
                
                if data_len != 1:
                    # 展示图像
                    ax[j].imshow(f_np)
                    ax[j].axis('off')
                else:
                    ax.imshow(f_np)
                    ax.axis('off')
            
                # 打印正确的标签
                if label != None:
                    print("Correct label: ", class_names[label[j]])
                print('Predict label: ', class_names[pre_label[j]])
                print('-----------------------------------\n\n')
                                
            # 将预测数据与真实数据加入各自集合中
            if label !=None:
                true_labels.append(label[j].item())
                pred_ans.append(1 if pre_label[j].item()==label[j].item() else 0)
            pred_labels.append(pre_label[j].item())
            
        plt.show()
    # 计算正确率
    if label!=None:
        correct = (100*pre_correct/(length*data_len))
        if prt_info: # 要输出信息时
            print('\n\ncorrect: %.3f%%' % correct)
    return pred_labels, true_labels, pred_ans

In [ ]:
predict(model_qye, test_set, 5)

In [ ]:
predict(model_qye, valid_set, 5)

In [ ]:
import csv
def save_predict_csv(model, data_set, length, name_row, write_path, file_names=None):
    preds, labels, pred_ans = predict(model, data_set, length, False)
    if file_names != None:
        ans = np.array([preds, file_names[0:length]]).T
    else:
        ans = np.array([preds, labels, pred_ans]).T
    with open(write_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(name_row)
        i = 0
        for row in ans:
            writer.writerow(row)
            i+=1
        print(f'Write over {i} lines in file {write_path}')

In [ ]:
save_predict_csv(model_qye, test_set, len(test_set), ['pred', 'filename'], '/kaggle/working/ans_ours.csv', test_pic_names)

In [ ]:
save_predict_csv(model_qye, valid_set, len(valid_set), ['pred', 'true', 'ans'], '/kaggle/working/valid_ours.csv')

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def draw_confusion_matrix(label_true, label_pred, label_name, title="Confusion Matrix", pdf_save_path=None, dpi=100):
    cm = confusion_matrix(y_true=label_true, y_pred=label_pred, normalize='true')

    fig = plt.figure(figsize=(8, 8))  # 设置图像大小
    plt.imshow(cm, cmap='Blues')
    plt.title(title)
    plt.xlabel("Predict label")
    plt.ylabel("Truth label")
    plt.yticks(range(len(label_name)), label_name)
    plt.xticks(range(len(label_name)), label_name, rotation=90)

    plt.tight_layout()

    plt.colorbar()

    for i in range(len(label_name)):
        for j in range(len(label_name)):
            color = (1, 1, 1) if i == j else (0, 0, 0)  
            value = float(format('%.2f' % cm[j, i]))
            plt.text(i, j, value, verticalalignment='center', horizontalalignment='center', color=color)

    if pdf_save_path is not None:
        plt.savefig(pdf_save_path, bbox_inches='tight', dpi=dpi)

    plt.show()  # 显示图像

In [ ]:
import pandas as pd
data = pd.read_csv('/kaggle/working/valid_ours.csv')
pred_lst = data['pred']
true_lst = data['true']
ans_lst = data['ans']

In [ ]:
# 训练集混淆矩阵
draw_confusion_matrix(label_true=true_lst,
                      label_pred=pred_lst,
                      label_name=class_names,
                      title='Validation Dataset Confusion Matrix',
                      pdf_save_path='/kaggle/working/valid.png',
                      dpi=1000
                     )

In [ ]:
accuarcy = 100*(sum(ans_lst)*1.0/len(ans_lst))
print(f'Eval accuarcy: {accuarcy:.3f} %')

## 搭建vgg19_bn模型并打印模型参数

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torchsummary import summary

class my_VGG19(nn.Module):
    def __init__(self):
        super(my_VGG19, self).__init__()
        self.conv_pool1 = nn.Sequential(
            # 输入 224*224*3
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # 输出 [(224-3+2)/1]+1 = 224
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # 输出224 224 64
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出(224-2)/2 +1 = 112 112 64
        )
        self.conv_pool2 = nn.Sequential(
            # 输入 112 112 64
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            # 输出 112-3+2+1 = 112
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            # 输出 112 112 128
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出 56 56 128
        )
        self.conv_pool3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # 输出 56-3+2+1=56
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            # 输出 56
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出 28 28 256
        )
        self.conv_pool4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            # 输出28 28 512
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出14 14 512
        )
        self.conv_pool5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            # 输出 14 14 512
            nn.MaxPool2d(kernel_size=2, stride=2)
            # 输出 7 7 512
        )
        self.fc = nn.Sequential(
            nn.Linear(7*7*512, 4096),
            nn.Dropout(), #0.5概率神经元失活
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 11)
            # 使用交叉熵误差函数，最后一层不用softmax
        )
    def forward(self, x):
        x = self.conv_pool1(x)
        x = self.conv_pool2(x)
        x = self.conv_pool3(x)
        x = self.conv_pool4(x)
        x = self.conv_pool5(x)
        x = x.view(-1, 7*7*512)
        x = self.fc(x)
        return x

model_vgg = my_VGG19().to(dvc)
summary(model_vgg, (3, 224, 224))

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from datetime import timedelta
from datetime import timezone
import time
from torch.optim.lr_scheduler import StepLR

SHA_TZ = timezone(
    timedelta(hours=8),
    name='Asia/Shanghai',
)

def get_cos(t):
    return 0.000015 + 0.00001499*math.cos(2*math.pi*t/10)


def vgg_train_valid(model, train_set, valid_set):
    writer = SummaryWriter('/kaggle/working/runs/vgg') # 开始记录
    loss_func = nn.CrossEntropyLoss() #交叉熵损失
    epochs = 100
    good_num = 0
    start_time = time.time()
    for epoch in range(epochs):
        lr = get_cos(epoch)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr) #优化器以及学习率
        if good_num >= 5:
            print(f'run {epoch} epoch')
            break
        model.train()
        sum_loss = 0
        train_correct = 0
        time_now = datetime.utcnow().astimezone(SHA_TZ)
        print(f'[{epoch+1}/{epochs}]  time now: {time_now.strftime("%Y-%m-%d %H:%M:%S")}')
        for i, data in enumerate(train_set, 0):
            features, labels = data
            data_len = len(data[0])
            features, labels = features.to(dvc), labels.to(dvc)
            preds = model(features)
            model.zero_grad() #梯度清零
            # 计算损失
            loss = loss_func(preds, labels)
            loss.backward() # 反向传播
            optimizer.step()
            sum_loss += loss.data
            _, ans = torch.max(preds, 1) # 此时preds第0维度有5个数据，ans也是5*1的张量
            train_correct += torch.sum(ans==labels.data) # ans5*1 ，这里输出的是5个里面预测准确的数量
        train_ac = 100*train_correct/(len(train_set)*data_len) # 训练集准确率
        train_loss = 100*sum_loss/(len(train_set)*data_len) # 训练集损失
        print(f'Loss on train set: {train_loss:.3f} Accuracy on train set: {train_ac:.3f}%')
        valid_ac = valid(model, valid_set) # 测试集准确率(有输出)
        if valid_ac > 90 or train_ac > 99:
            good_num += 1
        elif valid_ac == 95:
            good_num = 5
        writer.add_scalar("Accuracy/train", train_ac, epoch)
        writer.add_scalar("Loss/train", train_loss, epoch)
        writer.add_scalar("Accuracy/valid", valid_ac, epoch)
    end_time = time.time()
    print(f'cost time {end_time - start_time} s')
    writer.close() # 结束记录

In [ ]:
vgg_train_valid(model_vgg, train_set, valid_set)

In [ ]:
torch.save(model_vgg, '/kaggle/working/vggcos.pth')

## 对测试集进行预测，保存结果到ans_vgg.csv中

In [ ]:
save_predict_csv(model_vgg, test_set, len(test_set), ['pred', 'filename'], '/kaggle/working/ans_vgg.csv', test_pic_names)